In [6]:
from data_helpers import*
import xml.etree.ElementTree as ET
import numpy as np
from datetime import datetime
from matplotlib import pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import joblib
import os

input_file = 'datasets/raw/OhioT1DM/591-ws-testing.xml'
output_file = 'datasets/processed/591-test.csv'

In [7]:
root = ET.parse(input_file).getroot()

glucose_level = parse_data('glucose_level') # continuous data
finger_stick = parse_data('finger_stick') #sparse data
basal = parse_data('basal') #sparse data 
basis_heart_rate = parse_data('basis_heart_rate') #continuous data
basis_gsr = parse_data('basis_gsr') #continuous data
basis_skin_temperature = parse_data('basis_skin_temperature') #continuous data
basis_air_temperature = parse_data('basis_air_temperature') #continuous data
basis_steps = parse_data('basis_steps') #continuous data
temp_basal = parse_data('temp_basal') #Tb #Te #dose
bolus = parse_data('bolus') #Tb #Te #dose #carb_input



NameError: name 'parse_data' is not defined

In [ ]:
measurements = {
    'glucose_level': glucose_level,
    'finger_stick': finger_stick,
    'basal': basal,
    'basis_heart_rate': basis_heart_rate,
    'basis_gsr': basis_gsr,
    'basis_skin_temperature': basis_skin_temperature,
    'basis_air_temperature': basis_air_temperature,
    'basis_steps': basis_steps
}

# Convert each measurement array to a DataFrame with proper datetime conversion.
dfs = {}
global_min = None
global_max = None

for name, arr in measurements.items():
    df = pd.DataFrame(arr, columns=['timestamp', name])
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values('timestamp').reset_index(drop=True)
    dfs[name] = df
    
    # Update the global min and max timestamps.
    current_min = df['timestamp'].min()
    current_max = df['timestamp'].max()
    if global_min is None or current_min < global_min:
        global_min = current_min
    if global_max is None or current_max > global_max:
        global_max = current_max

# Force the global start to midnight.
global_start = global_min.normalize()
global_end = global_max

# Create a perfect timestamp index with a 5-minute frequency over the full range.
perfect_index = pd.date_range(start=global_start, end=global_end, freq='5min')
df_perfect = pd.DataFrame({'perfect_timestamp': perfect_index})



# Start with the perfect index DataFrame and then add new columns for each measurement.
df_final = df_perfect.copy()

for meas, df_meas in dfs.items():
    # Merge the measurement into the perfect index.
    df_merge = merge_measurement(df_final[['perfect_timestamp']], df_meas, meas)
    
    # Apply the function to choose the nearest timestamp and compute the time difference.
    res = df_merge.apply(lambda row: pd.Series(choose_nearest(row)), axis=1)
    df_merge[f'{meas}_associated_timestamp'] = res.iloc[:, 0]
    df_merge[f'{meas}_time_diff_min'] = res.iloc[:, 1]
    
    # Choose the measurement value using the threshold condition.
    df_merge[f'{meas}_final'] = df_merge.apply(lambda row: choose_value(row, meas), axis=1)
    
    # Add the measurement columns to the final DataFrame.
    df_final[meas] = df_merge[f'{meas}_final']
    df_final[f'{meas}_time_diff_min'] = df_merge[f'{meas}_time_diff_min']


# Forward fill the basal values.
df_final['basal'] = df_final['basal'].ffill()
# Backward fill the basal values (for the first few rows).
df_final['basal'] = df_final['basal'].bfill()

In [16]:

# Create the real_basal column as a copy of the forward-filled basal values.
df_final['real_basal'] = df_final['basal'].copy()

# For each temp_basal episode, overwrite real_basal in that time interval.
for row in temp_basal:
    t_start = pd.to_datetime(row[0])
    t_end   = pd.to_datetime(row[1])
    val     = row[2]
    mask = (df_final['perfect_timestamp'] >= t_start) & (df_final['perfect_timestamp'] <= t_end)
    df_final.loc[mask, 'real_basal'] = val


df_final['bolus'] = 0.0

# Process each bolus entry.
for row in bolus:
    t_beg = pd.to_datetime(row[0])
    t_end = pd.to_datetime(row[1])
    dose  = float(row[2])
    
    if t_beg == t_end:
        # Single data point: find the perfect timestamp that is closest.
        diffs = (df_final['perfect_timestamp'] - t_beg).abs()
        idx = diffs.idxmin()
        # Optionally, you can decide to only assign if the closest point is within a threshold.
        df_final.loc[idx, 'bolus'] += dose
    else:
        # Distributed bolus: find all perfect timestamps in the interval.
        mask = (df_final['perfect_timestamp'] >= t_beg) & (df_final['perfect_timestamp'] <= t_end)
        count = mask.sum()
        if count > 0:
            distributed = dose / count
            df_final.loc[mask, 'bolus'] += distributed


df_with_iob = compute_iob(df_final)


In [17]:
df= df_with_iob.drop(columns=['glucose_level_time_diff_min','finger_stick', 'finger_stick_time_diff_min', 'basal',
       'basal_time_diff_min', 'basis_heart_rate_time_diff_min', 'basis_gsr',
       'basis_gsr_time_diff_min', 'basis_skin_temperature','basis_skin_temperature_time_diff_min',
       'basis_air_temperature','basis_air_temperature_time_diff_min', 'basis_steps','basis_steps_time_diff_min'],inplace=False)

In [ ]:
# Derivatives
df['glucose_derivative'] = df['glucose_level'].diff()/5  # Difference from previous
df['heart_rate_derivative'] = df['basis_heart_rate'].diff()/5

# Function to calculate trend (slope over last 30 mins = 6 points)
def compute_trend(series, window=6):
    trend = []
    for i in range(len(series)):
        if i < window - 1 or series[i - window + 1:i + 1].isna().any():
            trend.append(np.nan)
        else:
            y = series[i - window + 1:i + 1].values.reshape(-1, 1)
            x = np.arange(window).reshape(-1, 1)
            model = LinearRegression().fit(x, y)
            trend.append(model.coef_[0][0])
    return trend


# Trend columns
df['glucose_trend'] = compute_trend(df['glucose_level'])
df['heart_rate_trend'] = compute_trend(df['basis_heart_rate'])




In [ ]:
def compute_tia(df, DOA_hours=5, interval_min=5):
    """
    Compute the Total Insulin Action (TIA) for each row of the DataFrame.
    
    Parameters:
        df (pd.DataFrame): DataFrame with columns 'real_basal' (in U/hr) and 'bolus' (in U).
        DOA_hours (float): Duration of insulin action in hours (how long a bolus affects TIA).
        interval_min (float): Time difference between consecutive rows in minutes.
    
    Returns:
        pd.DataFrame: A new DataFrame with added 'TIA' column (U/hr).
    """
    # Convert time step to hours
    interval_hr = interval_min / 60.0
    
    # Number of steps for insulin decay
    decay_steps = int(DOA_hours / interval_hr)
    
    # Define decay curve (quadratic) and normalize to sum=1
    t = np.linspace(0, 1, decay_steps)
    decay = 1 - (t**2) * (3 - 2*t)  # Quadratic decay from 1 to 0
    decay /= decay.sum()  # Normalize so total decayed bolus = original amount
    
    # Compute bolus contributions over time using convolution
    bolus_contrib = np.convolve(df['bolus'], decay, mode='full')[:len(df)]
    
    # Convert bolus contribution to U/hr (since interval is in minutes)
    bolus_contrib_hr = bolus_contrib / (interval_min / 60)
    
    # Total Insulin Action = Basal (U/hr) + Bolus Contribution (U/hr)
    df_out = df.copy()
    df_out['TIA'] = df['real_basal'] + bolus_contrib_hr
    
    return df_out

In [ ]:
basal_min, basal_max = 0.0, 2.0  # Example: Basal ranges from 0 to 2 U/hr
bolus_min, bolus_max = 0.0, 25.0   # Example: Bolus ranges from 0 to 25 U
df["basal"] = (df["basal"] - basal_min) / (basal_max - basal_min) * 2 - 1
df["bolus"] = (df["bolus"] - bolus_min) / (bolus_max - bolus_min) * 2 - 1

In [21]:
df.rename(columns={
    "perfect_timestamp":   "time",
    "glucose_level":       "glu",
    "basis_heart_rate":    "hr",
    "real_basal":          "basal",
    "IOB":                 "iob",
    "glucose_derivative":  "glu_d",
    "heart_rate_derivative": "hr_d",
    "glucose_trend":       "glu_t",
    "heart_rate_trend":    "hr_t"
}, inplace=True)

df["glu_raw"] = df["glu"]

features_to_normalize = [
    "glu", "glu_d", "glu_t",
    "hr",  "hr_d",  "hr_t", "iob"]



scaler = StandardScaler()
df[features_to_normalize] = scaler.fit_transform(df[features_to_normalize])
joblib.dump(scaler,'scaler.pkl')



['scaler.pkl']

In [30]:

# Add hour and hour_norm
df["hour_day"] = pd.to_datetime(df["time"]).dt.hour
df["hour"] = df["hour_day"] / 24.0

# Define episodes by day (daily episodes)
df["day"] = pd.to_datetime(df["time"]).dt.date
df["done"] = (df["day"] != df["day"].shift(-1)).astype(int)

# Drop helper columns if needed
df.drop(columns=["hour_day", "day"], inplace=True)

# Reorder columns
ordered_columns = [
    "time", # Optional timestamp for reference
    "glu_raw",            
    "glu", "glu_d", "glu_t",
    "hr", "hr_d", "hr_t",
    "iob", "hour",  # <- state features
    "basal", "bolus",      # <- action features
    "done"               # <- episode boundary flag
]

df = df[ordered_columns]




In [ ]:

df.to_csv(output_file, index=False)



